In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import sklearn
from sklearn import model_selection

Using TensorFlow backend.


In [8]:
df=pd.read_csv('../input/train.csv').values
print(df)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [7 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]]


In [9]:
X_train = df[:35000, 1:].reshape(35000,28, 28,1).astype( 'float32' )
X_test=df[35001:,1:].reshape(df.shape[0]-35000-1,28, 28,1).astype( 'float32' )
X_train = X_train/ 255.0
X_test=X_test/255.0
Y_train = df[:35000,0]
Y_test=df[35001:,0]

In [10]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)

In [11]:
def cnn(input_shape): 
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    X = Conv2D(64, (5, 5), padding='valid',strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv2')(X)
    X = Activation('relu')(X)
    X=MaxPooling2D((5, 5), name='max_pool2')(X)
    X = Flatten()(X)
    #Three fully connected layers
    X=Dense(10000, activation='relu', name='fc1',kernel_regularizer=regularizers.l2(0.01))(X)
    X = Dense(100, activation='relu', name='fc2',kernel_regularizer=regularizers.l2(0.01))(X)
    X = Dense(10, activation='relu', name='fc3')(X)
    
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    return model

In [12]:
from keras import regularizers
model = cnn(X_train[1].shape)
model.compile(optimizer = 'Adam', loss ="mean_squared_error", metrics = ["accuracy"])
model.fit(x =X_train, y =Y_train, epochs = 1, batch_size = 16)

Epoch 1/10
35000/35000 [==============================] - 313s 9ms/step - loss: 0.0709 - acc: 0.8971
Epoch 2/10
26400/35000 [=====================>........] - ETA: 1:15 - loss: 0.0198 - acc: 0.9717

KeyboardInterrupt: 

In [17]:
preds =model.evaluate(x = X_test, y = Y_test)
print(preds)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

6999/6999 [==============================] - 10s 1ms/step
[0.016147999806165405, 0.9742820402914703]
Loss = 0.016147999806165405
Test Accuracy = 0.9742820402914703


ValueError: cannot reshape array of size 21924000 into shape (28000,28,28,1)